In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Added required data sets 
df=pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")
test_df_original=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
train_df_original=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

# **Find Missing Data**

In [ ]:
#Train Data
train_df_modified=train_df_original.drop("Id",axis=1)
#train_df_modified.isnull().sum().sort_values()[61:]
per_nan_train=(train_df_modified.isnull().sum().sort_values()/len(train_df_modified))*100
#per_nan_train[61:]

#Test Data
test_df_modified=test_df_original.drop("Id",axis=1)
per_nan_test=(test_df_modified.isnull().sum().sort_values()/len(test_df_modified))*100
#per_nan_test[61:]

# **Treat Missing Data**

In [ ]:
#Train Data
train_df_modified=train_df_modified.dropna(axis=0,subset=["Electrical"])
train_df_modified["MasVnrType"]=train_df_modified["MasVnrType"].fillna("None")
train_df_modified["MasVnrArea"]=train_df_modified["MasVnrArea"].fillna(0)
bsmt_cols=["BsmtFinType1","BsmtCond","BsmtQual","BsmtExposure","BsmtFinType2"]
train_df_modified[bsmt_cols]=train_df_modified[bsmt_cols].fillna('NA')
Garage_cols=["GarageQual","GarageFinish","GarageCond","GarageType"]
train_df_modified[Garage_cols]=train_df_modified[Garage_cols].fillna("None")
train_df_modified["GarageYrBlt"]=train_df_modified["GarageYrBlt"].fillna(0)
train_df_modified["FireplaceQu"]=train_df_modified["FireplaceQu"].fillna("None")
train_df_modified["LotFrontage"]=train_df_modified.groupby("Neighborhood")["LotFrontage"].transform(lambda val: val.fillna(val.mean()))
train_df_modified=train_df_modified.drop(["MiscFeature","Alley","Fence","PoolQC"],axis=1)

#Test Data
test_df_modified=test_df_modified.dropna(axis=0,subset=["Electrical"])
test_df_modified["MasVnrType"]=test_df_modified["MasVnrType"].fillna("None")
test_df_modified["MasVnrArea"]=test_df_modified["MasVnrArea"].fillna(0)
bsmt_cols_test=["BsmtFinType1","BsmtCond","BsmtQual","BsmtExposure","BsmtFinType2"]
test_df_modified[bsmt_cols_test]=test_df_modified[bsmt_cols_test].fillna('NA')
Garage_cols_test=["GarageQual","GarageFinish","GarageCond","GarageType"]
test_df_modified[Garage_cols_test]=test_df_modified[Garage_cols_test].fillna("None")
test_df_modified["GarageYrBlt"]=test_df_modified["GarageYrBlt"].fillna(0)
test_df_modified["FireplaceQu"]=test_df_modified["FireplaceQu"].fillna("None")
test_df_modified["LotFrontage"]=test_df_modified.groupby("Neighborhood")["LotFrontage"].transform(lambda val: val.fillna(val.mean()))
test_df_modified=test_df_modified.drop(["MiscFeature","Alley","Fence","PoolQC"],axis=1)
test_df_modified[test_df_modified["GarageArea"].isnull()]
bsmt_cols_num=["TotalBsmtSF","BsmtUnfSF","BsmtFinSF2","BsmtFinSF1","BsmtFullBath","BsmtHalfBath"]
test_df_modified[bsmt_cols_num]=test_df_modified[bsmt_cols_num].fillna(0)
Garage_cols_num=["GarageArea","GarageCars"]
test_df_modified[Garage_cols_num]=test_df_modified[Garage_cols_num].fillna(0)



# **Dealing with Categorial Data**

In [ ]:
#Train Data
train_df_objs_modified=train_df_modified.select_dtypes(include="object")
#train_df_objs_modified.info()

#Test Data
test_df_objs_modified=test_df_modified.select_dtypes(include="object")
#test_df_objs_modified.info()


In [ ]:
#To Check Unwanted Categorial Data

import matplotlib.pyplot as plt
import seaborn as sns
fig, axes = plt.subplots(
    round(len(train_df_objs_modified.columns) /3 ), 3, figsize=(12, 30))

for i, ax in enumerate(fig.axes):
    
    #if i < len(train_df_objs_modified.columns) - 1:
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
        sns.countplot(
            x=train_df_objs_modified.columns[i], alpha=0.5, data=train_df_objs_modified, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
#To Check Unwanted Categorial Data

sns.countplot(x="SaleType",data=train_df_objs_modified)
train_df_objs_modified["SaleType"].value_counts()
#train_df_objs_modified.columns

In [ ]:
#Train Data
train_df_objs_modified=train_df_objs_modified.drop(["Street","Utilities","Condition2","Heating","Functional","RoofMatl"],axis=1)

#Test Data
test_df_objs_modified=test_df_objs_modified.drop(["Street","Utilities","Condition2","Heating","Functional","RoofMatl"],axis=1)

In [ ]:
#Train Data
train_df_objs_modified=pd.get_dummies(train_df_objs_modified,drop_first=True)
#to remove columns with most of the values are 0
cols=list(train_df_objs_modified.columns)
cols_obj=[]
for i in cols:
    if train_df_objs_modified[i].value_counts()[0]>1445:
        cols_obj.append(i)
train_df_objs_modified=train_df_objs_modified.drop(cols_obj,axis=1)

cols_missmatch=["GarageQual_Fa","GarageQual_Gd","HouseStyle_1.5Unf","MSZoning_RH"]
train_df_objs_modified=train_df_objs_modified.drop(cols_missmatch,axis=1)

#test Data
test_df_objs_modified=pd.get_dummies(test_df_objs_modified,drop_first=True)
#to remove columns with most of the values are 0
cols=list(test_df_objs_modified.columns)
cols_obj=[]
for i in cols:
    if test_df_objs_modified[i].value_counts()[0]>1445:
        cols_obj.append(i)               
test_df_objs_modified=test_df_objs_modified.drop(cols_obj,axis=1)

cols_missmatch=["Condition1_RRAe","Exterior2nd_Brk Cmn","Neighborhood_NPkVill","SaleType_ConLD"]
test_df_objs_modified=test_df_objs_modified.drop(cols_missmatch,axis=1)


# **Combine Numerical and Categorial Features**

In [ ]:
#Train Data
train_df_num_modified=train_df_modified.select_dtypes(exclude="object")
train_df_modified=train_df_num_modified.join(train_df_objs_modified)
#train_df_modified.info()

#Test Data
test_df_num_modified=test_df_modified.select_dtypes(exclude="object")
test_df_modified=test_df_num_modified.join(test_df_objs_modified)
#test_df_modified.info()

# **Outliers in Numerical Data**

In [ ]:
train_df_num_modified.corr()["SalePrice"].sort_values()

In [ ]:
#to check outliers
sns.scatterplot(x="YearRemodAdd",y="SalePrice",data=train_df_modified)

# **Remove Outliers**

In [ ]:
ind=train_df_num_modified[(train_df_num_modified["GrLivArea"]>4000)&(train_df_num_modified["SalePrice"]<300000)].index
train_df_modified=train_df_modified.drop(ind)

test_df_modified=test_df_modified.drop(ind)

In [ ]:
ind=train_df_modified[(train_df_modified["TotalBsmtSF"]>3000)&(train_df_modified["SalePrice"]<600000)].index
train_df_modified=train_df_modified.drop(ind)

test_df_modified=test_df_modified.drop(ind)

In [ ]:
ind=train_df_modified[(train_df_modified["GarageArea"]>1200)&(train_df_modified["SalePrice"]<300000)].index
train_df_modified=train_df_modified.drop(ind)

test_df_modified=test_df_modified.drop(ind)

In [ ]:
ind=train_df_modified[(train_df_modified["1stFlrSF"]>3000)&(train_df_modified["SalePrice"]<600000)].index
train_df_modified=train_df_modified.drop(ind)

test_df_modified=test_df_modified.drop(ind)

In [ ]:
ind=train_df_modified[(train_df_modified["SalePrice"]>700000)].index
train_df_modified=train_df_modified.drop(ind)

test_df_modified=test_df_modified.drop(ind)

# ** Simple Regression**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

X=train_df_modified.drop("SalePrice",axis=1)
y=train_df_modified["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
Linear_Regession=LinearRegression()
Linear_Regession.fit(X_train,y_train)
linear_regression_prediction=Linear_Regession.predict(X_test)
rmsesm=float(format(np.sqrt(metrics.mean_squared_error(y_test,linear_regression_prediction)),'.3f'))
rmsesm

In [ ]:
plt.figure()
plt.title("Actual vs. Predicted house prices\n ", fontsize=20)
plt.scatter(x=linear_regression_prediction,y=y_test,
            color="deepskyblue", marker="o", facecolors="none")
plt.plot([0, 800000], [0, 800000], "darkorange", lw=2)
plt.xlim(0, 800000)
plt.ylim(0, 800000)
plt.xlabel("\nActual Price", fontsize=16)
plt.ylabel("Predicted Price\n", fontsize=16)
plt.show()

# **Elastic Net Regression**

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

base_elastic_model = ElasticNet(max_iter=20000)

param_grid = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}

grid_model = GridSearchCV(estimator=base_elastic_model,
                          param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          verbose=1)

grid_model.fit(X_train,y_train)

y_pred = grid_model.predict(X_test)
rmsesm=float(format(np.sqrt(metrics.mean_squared_error(y_test,y_pred)),'.3f'))
rmsesm

In [ ]:
plt.figure()
plt.title("Actual vs. Predicted house prices\n (Elastic Net)", fontsize=20)
plt.scatter(x=y_pred,y=y_test,
            color="deepskyblue", marker="o", facecolors="none")
plt.plot([0, 800000], [0, 800000], "darkorange", lw=2)
plt.xlim(0, 800000)
plt.ylim(0, 800000)
plt.xlabel("\nActual Price", fontsize=16)
plt.ylabel("Predicted Price\n", fontsize=16)
plt.show()

# **Apply Finalised Model on Test Data**

In [ ]:
test_df_modified_final=scaler.transform(test_df_modified)
test_df_modified_final.size
X_test.size
y_pred = grid_model.predict(test_df_modified_final)
y_pred